In [1]:
import json
from pprint import pprint
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
healthy_fats = [
    "olive oil", "avocado", "canola oil", "peanut oil", "almond", "peanut",
    "cashew", "macadamia", "flaxseed", "chia seed", "sunflower oil",
    "soybean oil", "corn oil", "walnut", "salmon", "tuna", "mackerel", "sardine", "trout"
]

sugar_keywords = [
    "sugar"
]

sodium_keywords = [
    "salt", "sea salt", "sodium", "monosodium glutamate", "MSG", "soy sauce",
    "teriyaki", "brine", "cured", "pickled", "smoked", "bacon", "ham", "sausage",
    "pepperoni", "salami", "cheese", "bouillon", "broth", "stock"
]

fruits = [
    "apple", "banana", "orange", "grapefruit", "grape", "pear", "peach", "plum", "nectarine",
    "mango", "papaya", "pineapple", "kiwi", "pomegranate", "melon", "cantaloupe", "honeydew",
    "watermelon", "cherry", "blueberry", "strawberry", "raspberry", "blackberry", "cranberry",
    "fig", "date", "apricot", "lime", "lemon", "coconut", "passion fruit", "dragon fruit", "fruit"
]

vegetables = [
    "spinach", "kale", "lettuce", "romaine", "arugula", "chard", "collard", "mustard greens",
    "mixed greens", "mesclun", "spring mix", "salad greens", "broccoli", "cauliflower", "cabbage",
    "brussels sprouts", "bok choy", "carrot", "beet", "turnip", "parsnip", "radish", "vegetable",
    "sweet potato", "yam", "potato", "zucchini", "squash", "pumpkin", "eggplant"
]

desserts = [
  "cake", "cupcake", "cookie", "brownie", "ice cream", "sundae", "pudding", "mousse",
  "cheesecake", "donut", "doughnut", "pie", "tart", "cobbler", "crisp", "crumble",
  "frosting", "glaze", "batter", "fudge", "sorbet", "gelato", "popsicle",
  "macaron", "macaroon", "pastry", "eclair", "cream puff", "churro", "sweet roll",
  "danish", "strudel", "trifle", "shortcake", "meringue", "bread pudding", "sweet bread",
  "caramel", "toffee", "marshmallow", "nougat", "sugar cookie", "snickerdoodle",
  "biscotti", "molten", "lava cake", "lemon bar", "chocolate", "vanilla", "dessert"
]

def protein_density(menu_item):
    if menu_item['protein_g'] == 0 or menu_item['calories'] == 0:
        return 0
    return menu_item['protein_g'] / menu_item['calories']

def fat_density(menu_item):
    if menu_item['fat_g'] == 0 or menu_item['calories'] == 0:
        return 0
    return menu_item['fat_g'] / menu_item['calories']

def count_keywords(menu_item, keywords):
    count = 0
    current_index = 0

    for word in keywords:
        while True:
            loc = menu_item['ingredients'].lower().find(word, current_index)
            if loc != -1:
                count += 1
                current_index = loc + len(word)
            else:
                break

    return count

def is_fruit(menu_item):
    for fruit in fruits:
        if menu_item['item_name'].lower().find(fruit) != -1:
            return True
    return False

def is_vegetable(menu_item):
    for vegetable in vegetables:
        if menu_item['item_name'].lower().find(vegetable) != -1:
            return True
    return False

def is_dessert(menu_item):
    for dessert in desserts:
        if menu_item['item_name'].lower().find(dessert) != -1:
            return True
    return False

In [52]:
df = pd.read_csv('./test/menu_items_rows-4.csv').fillna('')

df = df.drop(columns=['created_at'])
# df = df.drop_duplicates(subset=['item_name'], keep='first')


df = df[(df['date'] == '2025-08-16') & (df['dining_hall'] == 'tercero') & (df['meal'] == 'lunch')]

for col in ['calories', 'protein_g', 'fat_g', 'carbohydrates_g']:
    df[col] = pd.to_numeric(df[col])

df['protein_density'] = df.apply(lambda x: protein_density(x), axis=1)
df['fat_density'] = df.apply(lambda x: fat_density(x), axis=1)
df['healthy_fats_count'] = df.apply(lambda x: count_keywords(x, healthy_fats), axis=1)
df['sugar_count'] = df.apply(lambda x: count_keywords(x, sugar_keywords), axis=1)
df['sodium_count'] = df.apply(lambda x: count_keywords(x, sodium_keywords), axis=1)
df['is_fruit'] = df.apply(lambda x: is_fruit(x), axis=1)
df['is_vegetable'] = df.apply(lambda x: is_vegetable(x), axis=1)
df['is_dessert'] = df.apply(lambda x: is_dessert(x), axis=1)

df = df.sort_values(by='protein_density', ascending=False)[['item_name', 'protein_density', 'fat_density', 'healthy_fats_count', 'sugar_count', 'sodium_count', 'is_fruit', 'is_vegetable', 'is_dessert']]

# print(len(df))

df = df[
    ((df['protein_density'] >= 0.05) |
    (((df['is_fruit'] == True) | (df['is_vegetable'] == True)) & (df['protein_density'] >= 0.01))) &
    (df['is_dessert'] == False)
]

df

# print(len(df))

# df.sort_values(by='sugar_count', ascending=False)

# df[df['is_fruit'] == True]

,item_name,protein_density,fat_density,healthy_fats_count,sugar_count,sodium_count,is_fruit,is_vegetable,is_dessert
1046,Sliced Smoked Turkey,0.200589,0.009937,0,1,0,False,False,False
1048,Sliced Roast Beef,0.156989,0.028520,0,0,1,False,False,False
1047,Sliced Smoked Ham,0.150148,0.033257,0,1,3,False,False,False
1007,Sautéed Chicken with Garlic and Onion,0.131245,0.042928,2,0,0,False,False,False
1050,Sliced Salami,0.128668,0.056998,0,2,3,False,False,False
1032,Tofu,0.119831,0.060505,0,0,0,False,False,False
1008,Pork Carnitas,0.099112,0.023598,0,0,1,False,False,False
1041,Swiss Cheese,0.074993,0.087492,0,0,0,False,False,False
1010,Chopped Romaine Lettuce,0.071721,0.018443,0,0,0,False,True,False
1044,Sliced Provolone Cheese,0.071504,0.085805,0,0,0,False,False,False


In [5]:
from config import *
import json

In [ ]:
def get_user_preferences(user_id):
    user_preferences = supabase_client.table('meal_preferences') \
        .select('*') \
        .eq('user_id', user_id) \
        .execute().data[0]
    return user_preferences


user_id = '6e3a820d-e976-483d-9df2-6402bcfe78a0'

user_preferences = get_user_preferences(user_id)

user_preferences

{'created_at': '2025-08-18T18:21:50.139675+00:00',
 'user_id': '6e3a820d-e976-483d-9df2-6402bcfe78a0',
 'calories': 2500,
 'protein': 120,
 'meals': {'lunch': 'tercero', 'dinner': 'tercero', 'breakfast': 'tercero'},
 'diets': [],
 'allergens': [],
 'meal_plan': None}

In [7]:
def contains_allergen(item, exclude_allergens):
    item_allergens = ', '.join(item['allergens'])
    for allergen in exclude_allergens:
        if item_allergens.find(allergen) != -1:
            return True
    return False

def filter_menu_items(date, user_preferences):
    filtered_menu_items = {}

    for meal in user_preferences['meals']:
        filtered_menu_items[meal] = []
        meal_items = supabase_client.table('menu_items') \
            .select('*') \
            .eq('date', date) \
            .eq('meal', meal) \
            .eq('dining_hall', user_preferences['meals'][meal]) \
            .contains('diets', json.dumps(user_preferences['diets'])) \
            .not_.in_('calories', ['N/A', '0']) \
            .not_.in_('protein_g', ['N/A', '0']) \
            .execute().data

        # Filter out items that contain user's allergens
        for item in meal_items:
            if not contains_allergen(item, user_preferences['allergens']):
                filtered_menu_items[meal].append(item)

    return filtered_menu_items

filtered_menu_items = filter_menu_items('2025-08-18', user_preferences)

with open('filtered_menu_items.json', 'w') as f:
    json.dump(filtered_menu_items, f, indent=4)

In [169]:
sodium_keywords = [
    "salt", "sea salt", "sodium", "monosodium glutamate", "MSG", "soy sauce",
    "teriyaki", "brine", "cured", "pickled", "smoked", "bacon", "ham", "sausage",
    "pepperoni", "salami", "cheese", "bouillon", "broth", "stock"
]

lean_proteins = []

fruits = [
    "apple", "banana", "orange", "grapefruit", "grape", "pear", "peach", "plum", "nectarine",
    "mango", "papaya", "pineapple", "kiwi", "pomegranate", "melon", "cantaloupe", "honeydew",
    "watermelon", "cherry", "blueberry", "strawberry", "raspberry", "blackberry", "cranberry",
    "fig", "date", "apricot", "lime", "lemon", "coconut", "passion fruit", "dragon fruit", "fruit"
]

vegetables = [
  "spinach", "kale", "lettuce", "romaine", "arugula", "chard", "collard", "mustard greens",
  "mixed greens", "mesclun", "spring mix", "salad greens", "broccoli", "cauliflower", "cabbage",
  "brussels sprouts", "bok choy", "carrot", "beet", "turnip", "parsnip", "radish",
  "zucchini", "squash", "eggplant", "vegetable"
]

desserts = [
  "cake", "cupcake", "cookie", "brownie", "ice cream", "sundae", "pudding", "mousse",
  "cheesecake", "donut", "doughnut", "pie", "tart", "cobbler", "crisp", "crumble",
  "frosting", "glaze", "batter", "fudge", "sorbet", "gelato", "popsicle",
  "macaron", "macaroon", "pastry", "eclair", "cream puff", "churro", "sweet roll",
  "danish", "strudel", "trifle", "shortcake", "meringue", "bread pudding", "sweet bread",
  "caramel", "toffee", "marshmallow", "nougat", "sugar cookie", "snickerdoodle",
  "biscotti", "molten", "lava cake", "lemon bar", "chocolate", "vanilla", "dessert"
]

def count_keywords(ingredients, keywords):
    count = 0
    current_index = 0
    for word in keywords:
        while True:
            loc = ingredients.lower().find(word, current_index)
            if loc != -1:
                count += 1
                current_index = loc + len(word)
            else:
                break
    return count

def is_food_type(food_name, food_group):
    for item in food_group:
        if food_name.lower().find(item) != -1:
            return True
    return False

def remove_unhealthy_items(menu_items):
    flattened_menu_items = []

    for meal in menu_items:
        flattened_menu_items.extend(menu_items[meal])

    df = pd.DataFrame(flattened_menu_items)
    for col in ['calories', 'protein_g', 'fat_g', 'carbohydrates_g']:
        df[col] = pd.to_numeric(df[col])

    df['protein_density'] = df['protein_g'] / df['calories']
    df['sodium_count'] = df['ingredients'].apply(lambda x: count_keywords(x, sodium_keywords))
    df['is_fruit'] = df['item_name'].apply(lambda x: is_food_type(x, fruits))
    df['is_vegetable'] = df['item_name'].apply(lambda x: is_food_type(x, vegetables))
    df['is_dessert'] = df['item_name'].apply(lambda x: is_food_type(x, desserts))

    df = df[(df["protein_density"] >= 0.01) & (df['is_dessert'] == False)]

    """
    df = df[
        ((df['protein_density'] >= 0.05) |
        (((df['is_fruit'] == True) | (df['is_vegetable'] == True)) & (df['protein_density'] >= 0.01))) &
        (df['is_dessert'] == False)
    ]

    menu_items = {}

    for meal in df['meal'].unique():
        meal_items = df[df['meal'] == meal][['id', 'item_name', 'calories', 'fat_g', 'carbohydrates_g', 'protein_g']]
        menu_items[meal] = meal_items.to_dict(orient='records')

    return menu_items
    """
    
    return df

df = remove_unhealthy_items(filtered_menu_items)

#df = df[(df['protein_density'] >= protein_density_goal_lb) & (df['protein_density'] <= protein_density_goal_ub)]

df[df['meal'] == 'breakfast'].sort_values(by='protein_density', ascending=False)

,id,created_at,date,dining_hall,meal,platform,item_name,serving_size_oz,calories,fat_g,carbohydrates_g,protein_g,allergens,diets,ingredients,description,feature,protein_density,sodium_count,is_fruit,is_vegetable,is_dessert
110,6361,2025-08-17T12:43:54.864747+00:00,2025-08-17,tercero,breakfast,Go Live,Baked Honey Ham,2.95,124.79,4.00,3.86,18.40,[vinegar],[],Fresh hickory smoked boneless ham (boneless co...,"Honey and mustard glazed ham, baked until tender.",,0.147448,2,False,False,False
107,6358,2025-08-17T12:43:54.864747+00:00,2025-08-17,tercero,breakfast,Go Live,Sautéed Chicken with Garlic and Onion,4,256.01,10.99,4.11,33.60,[],[halal],"Fresh chicken thigh sliced halal, 1/2in Diced ...",Halal chicken thighs seasoned with fresh parsl...,,0.131245,2,False,False,False
114,6365,2025-08-17T12:43:54.864747+00:00,2025-08-17,tercero,breakfast,Go Live,Sliced Cremini Mushrooms,100 g,22.47,0.10,4.30,2.50,[],"[vegan, halal]",Sliced cremini mushrooms,,,0.111259,0,False,False,False
129,6380,2025-08-17T12:43:54.864747+00:00,2025-08-17,tercero,breakfast,Fresh Inspirations,Boiled Eggs,2,81.17,5.39,0.41,7.12,[egg],"[vegetarian, halal]",Whole large white eggs,,,0.087717,0,False,False,False
120,6371,2025-08-17T12:43:54.864747+00:00,2025-08-17,tercero,breakfast,Pacific Fusion,Bacon,2,283.94,20.28,0.00,24.34,[],[],Raw 10/12 sliced pork bacon (pork cured with w...,Freshly-baked crispy bacon.,,0.085722,4,False,False,False
108,6359,2025-08-17T12:43:54.864747+00:00,2025-08-17,tercero,breakfast,Go Live,Bacon,2,283.94,20.28,0.00,24.34,[],[],Raw 10/12 sliced pork bacon (pork cured with w...,Freshly-baked crispy bacon.,,0.085722,4,False,False,False
122,6373,2025-08-17T12:43:54.864747+00:00,2025-08-17,tercero,breakfast,Pacific Fusion,Bacon,2,283.94,20.28,0.00,24.34,[],[],Raw 10/12 sliced pork bacon (pork cured with w...,Freshly-baked crispy bacon.,,0.085722,4,False,False,False
121,6372,2025-08-17T12:43:54.864747+00:00,2025-08-17,tercero,breakfast,Pacific Fusion,Scrambled Egg,3.05,138.75,9.54,1.82,10.86,[egg],"[vegetarian, halal]","Liquid whole eggs (cage-free whole eggs, citri...",Softly scrambled cage-free eggs.,,0.078270,0,False,False,False
123,6374,2025-08-17T12:43:54.864747+00:00,2025-08-17,tercero,breakfast,Pacific Fusion,Scrambled Egg,3.05,138.75,9.54,1.82,10.86,[egg],"[vegetarian, halal]","Liquid whole eggs (cage-free whole eggs, citri...",Softly scrambled cage-free eggs.,,0.078270,0,False,False,False
106,6355,2025-08-17T12:43:54.864747+00:00,2025-08-17,tercero,breakfast,Go Live,Basic Omelet,4.48,238.81,18.48,2.55,15.33,"[egg, dairy, vinegar]","[vegetarian, halal]","Liquid whole eggs (cage-free whole eggs, citri...",,Salsa|| Cheese|| Ketchup,0.064193,0,False,False,False


In [186]:
# Step 1: Add the healthiest item
# Step 2: Add a random fruit if not already in the meal
# Step 3: Add a random vegetable if not already in the meal

"""
Vegetables:   35% cal, 10% protein
Protein:      30% cal, 70% protein
Whole Grains: 25% cal, 15% protein
Fruits:       10% cal, 5% protein

Step 1. Create the meal split
    - does_vegetables_exist()
    - does_protein_exist()
    - does_whole_grains_exist()
    - does_fruits_exist()
    - If one of them does not exist, adjust the aplit accordingly

Step 2. Create the meal
    - get_vegetable_item()
    - get_protein_item()
    - get_whole_grain_item()
    - get_fruit_item()
"""

def meal_plan(menu_items, user_preferences):

    def split_goals_by_meal(selected_meals, calorie_goal, protein_goal):
        default_split = {
            'breakfast': 0.30,
            'lunch': 0.35,
            'dinner': 0.35
        }

        missing_meals = set(default_split.keys()) - set(selected_meals)

        for meal in missing_meals:
            additional_split = round(default_split[meal] / len(selected_meals), 2)
            for selected_meal in selected_meals:
                default_split[selected_meal] += additional_split
            del default_split[meal]

        for meal in default_split:
            default_split[meal] = {
                'calories': round(calorie_goal * default_split[meal], 2),
                'protein': round(protein_goal * default_split[meal], 2)
            }

        return default_split


    calorie_goal = user_preferences['calories']
    protein_goal = user_preferences['protein']

    return split_goals_by_meal(['breakfast', 'dinner'], calorie_goal, protein_goal)

meal_plan(filtered_menu_items, user_preferences)

{'breakfast': {'calories': 1034.0, 'protein': 56.4},
 'dinner': {'calories': 1144.0, 'protein': 62.4}}

In [ ]:
selected_meals = list(user_preferences['meals'].keys())
selected_meals_str = ', '.join(selected_meals)

calorie_goal = user_preferences['calories']
protein_goal = user_preferences['protein']

prompt = f"""
Create the healthiest meal plan for {selected_meals_str} that meets {calorie_goal} calories and {protein_goal}g protein using these items:

{json.dumps(filtered_menu_items)}

Combine multiple items per meal or include multiple servings of the same item if it's a healthy choice and needed to meet calorie and protein goals.
For each meal, provide a brief explanation (less than 200 characters) of why it is the best choice compared to other options."""

meal_items_schema = {
    "type": "object",
    "properties": {
        "items": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name": {"type": "string"},
                    "quantity": {"type": "integer"},
                },
                "required": ["name", "quantity"]
            }
        },
        "explanation": {"type": "string"}
    },
    "required": ["items", "explanation"]
}

meals_schema = {meal: meal_items_schema for meal in selected_meals}

response = openai_client.chat.completions.create(
    model="gpt-5-nano",
    messages=[
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": json.dumps(meal_plan)},
    ],
    store=True,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "meal_recommendation",
            "schema": {
                "type": "object",
                "properties": meals_schema,
                "required": selected_meals,
                "additionalProperties": False
            }
        }
    }
)

TypeError: Completions.create() got an unexpected keyword argument 'conversation'

In [150]:
meal_plan = json.loads(response.choices[0].message.content)
meal_plan

{'lunch': {'items': [{'name': 'Santa Fe Chicken Salad', 'quantity': 1},
   {'name': 'Sautéed Chicken with Garlic and Onion', 'quantity': 1},
   {'name': 'Cilantro Rice', 'quantity': 1},
   {'name': 'Plain Cauliflower Rice', 'quantity': 1},
   {'name': 'Sliced Smoked Turkey', 'quantity': 1}],
  'explanation': 'Lean proteins with grains create a filling, nutrient-dense midday meal.'},
 'dinner': {'items': [{'name': 'Roasted Rosemary Garlic Chicken',
    'quantity': 1},
   {'name': 'Fresh Fruit Salad', 'quantity': 1}],
  'explanation': 'Protein-rich chicken with a light fruit side for recovery without excess fat.'},
 'breakfast': {'items': [{'name': 'Basic Omelet', 'quantity': 1},
   {'name': 'Scrambled Egg Whites', 'quantity': 1},
   {'name': 'Scrambled Egg', 'quantity': 1},
   {'name': 'Fresh Fruit Salad', 'quantity': 2}],
  'explanation': 'Eggs provide solid protein; fruit adds vitamins and fiber for a balanced morning.'}}

In [1]:
from meal_plan import MealPlanner
import pandas as pd


meal_planner = MealPlanner('2025-08-17', '6e3a820d-e976-483d-9df2-6402bcfe78a0')

In [2]:
meal_plans = meal_planner.plan_meals(max_repeated_items=3)

In [3]:
calories = 0
protein = 0

for meal, plan in meal_plans.items():
    print(meal)
    for item in plan:
        print(item['item']['item_name'], item['quantity'])
        calories += item['item']['calories'] * item['quantity']
        protein += item['item']['protein_g'] * item['quantity']
    print()

calories, protein

breakfast
Basic Omelet 1
Plain Bagel 1
Congee Rice Porridge 3
Fresh Fruit Salad 3
Overnight Oats 1
Oatmeal 2

lunch
Grass-Fed Cheeseburger 1
Nine Grain Bread 1
Chopped Romaine Lettuce 3
Watermelon 1
Sliced Genoa Salami 1
Mango Chunks 1

dinner
Grass-Fed Cheeseburger 1
Nine Grain Bread 1
Chopped Romaine Lettuce 3
Watermelon 1
Sliced Genoa Salami 1
Mango Chunks 1



(2171.2900000000004, 108.47000000000001)

In [4]:
cols = ['item_name', 'calories', 'protein_g', 'protein_density', 'pd_difference', 'is_fruit', 'is_vegetable', 'is_protein', 'is_lean_protein', 'is_grain', 'is_whole_grain']

meal = 'breakfast'
calorie_goal = 700
protein_goal = 42

menu_items = df[df['meal'] == meal]

selected_items = df.head(0).copy()

max_repeated_items = 2
i = 0
order = ['protein', 'grain', 'vegetable', 'fruit']

empty_groups = set()

def add_item(food_group):
    global selected_items
    
    selected_item_counts = selected_items['id'].value_counts()
    repeated_ids = selected_item_counts[selected_item_counts >= max_repeated_items].index

    filtered_items = menu_items[
        (menu_items[f'is_{food_group}'] == True) &
        (~menu_items['id'].isin(repeated_ids)) &
        (menu_items['calories'] < calorie_goal - sum(selected_items['calories'])) &
        (menu_items['protein_g'] < protein_goal - sum(selected_items['protein_g']))
    ]

    if food_group in ['protein', 'grain']:
        items_to_add = filtered_items[filtered_items['calories'] > 150]
    
    elif food_group in ['fruit', 'vegetable']:
        items_to_add = filtered_items[filtered_items['calories'] <= 150]

    if food_group in ['protein', 'grain'] and items_to_add.empty:
        items_to_add = filtered_items[filtered_items['calories'] <= 150]

    if items_to_add.empty:
        empty_groups.add(food_group)
    else:
        item_to_add = items_to_add.sort_values(by='pd_difference', ascending=True).iloc[0]
        selected_items = pd.concat([selected_items, pd.DataFrame([item_to_add])])

goals_met = lambda: sum(selected_items['calories']) >= calorie_goal and sum(selected_items['protein_g']) >= protein_goal

while not goals_met() and len(empty_groups) < 4:

    food_group = order[i % 4]

    if not selected_items[f'is_{food_group}'].any() or (i > 0 and not goals_met()):
        add_item(food_group)

    i += 1

print(f'Calories: {sum(selected_items["calories"])}')
print(f'Protein: {sum(selected_items["protein_g"])}')

selected_items

Calories: 687.8
Protein: 31.08


,id,created_at,date,dining_hall,meal,platform,item_name,serving_size_oz,calories,fat_g,...,feature,protein_density,pd_difference,is_fruit,is_vegetable,is_protein,is_lean_protein,is_grain,is_whole_grain,is_dessert
106,6355,2025-08-17T12:43:54.864747+00:00,2025-08-17,tercero,breakfast,Go Live,Basic Omelet,4.48,238.81,18.48,...,Salsa|| Cheese|| Ketchup,0.064193,0.009648,False,False,True,True,False,False,False
132,6383,2025-08-17T12:43:54.864747+00:00,2025-08-17,tercero,breakfast,Harvest Bakery,Plain Bagel,2,180.87,0.75,...,,0.036048,0.018497,False,False,False,False,True,True,False
128,6379,2025-08-17T12:43:54.864747+00:00,2025-08-17,tercero,breakfast,Fresh Inspirations,Congee Rice Porridge,1.31,19.22,0.36,...,,0.044225,0.010321,False,True,True,True,True,False,False
126,6377,2025-08-17T12:43:54.864747+00:00,2025-08-17,tercero,breakfast,Fresh Inspirations,Fresh Fruit Salad,100 g,34.00,0.14,...,,0.014706,0.039840,True,False,False,False,False,False,False
128,6379,2025-08-17T12:43:54.864747+00:00,2025-08-17,tercero,breakfast,Fresh Inspirations,Congee Rice Porridge,1.31,19.22,0.36,...,,0.044225,0.010321,False,True,True,True,True,False,False
132,6383,2025-08-17T12:43:54.864747+00:00,2025-08-17,tercero,breakfast,Harvest Bakery,Plain Bagel,2,180.87,0.75,...,,0.036048,0.018497,False,False,False,False,True,True,False
127,6378,2025-08-17T12:43:54.864747+00:00,2025-08-17,tercero,breakfast,Fresh Inspirations,Oatmeal,0.98,14.81,0.26,...,,0.034436,0.020109,False,False,False,False,True,True,False


In [2]:
meal_planner.plan_meal('breakfast', 700, 42)

NameError: name 'selected_items' is not defined

In [78]:
id_counts = selected_items['id'].value_counts()

# selected_items = selected_items.merge(id_counts, on='id', how='left')

selected_items.drop_duplicates(subset=['id'], inplace=True)

selected_items = selected_items.drop(['protein_density', 'pd_difference', 'is_fruit', 'is_vegetable', 'is_grain', 'is_protein', 'is_lean_protein', 'is_whole_grain', 'is_dessert'], axis=1)

selected_items = selected_items.to_dict('records')

In [79]:
for i in range(len(selected_items)):
    selected_items[i] = {
        'item': selected_items[i],
        'quantity': int(id_counts[selected_items[i]['id']])
    }

In [80]:
selected_items

[{'item': {'id': 6355,
   'created_at': '2025-08-17T12:43:54.864747+00:00',
   'date': '2025-08-17',
   'dining_hall': 'tercero',
   'meal': 'breakfast',
   'platform': 'Go Live',
   'item_name': 'Basic Omelet',
   'serving_size_oz': '4.48',
   'calories': 238.81,
   'fat_g': 18.48,
   'carbohydrates_g': 2.55,
   'protein_g': 15.33,
   'allergens': ['egg', 'dairy', 'vinegar'],
   'diets': ['vegetarian', 'halal'],
   'ingredients': 'Liquid whole eggs (cage-free whole eggs, citric acid), Canola and Olive Oil Blend 90/10 (canola oil, extra virgin olive oil)',
   'description': '',
   'feature': 'Salsa|| Cheese|| Ketchup'},
  'quantity': 1},
 {'item': {'id': 6379,
   'created_at': '2025-08-17T12:43:54.864747+00:00',
   'date': '2025-08-17',
   'dining_hall': 'tercero',
   'meal': 'breakfast',
   'platform': 'Fresh Inspirations',
   'item_name': 'Congee Rice Porridge',
   'serving_size_oz': '1.31',
   'calories': 19.22,
   'fat_g': 0.36,
   'carbohydrates_g': 3.12,
   'protein_g': 0.85,
   